In [ ]:
http://tianqi.2345.com/t/wea_history/js/56294_20112.js
# 导入第三方包
import requests
import time
import random
import pandas as pd
import re

# 生成请求头
headers = {}


urls = []
for year in range(2011,2020):
    for month in range(1,13):
        if year <= 2016:
            urls.append('http://tianqi.2345.com/t/wea_history/js/56294_%s%s.js' %(year,month))
        else:
            if month<10:
                urls.append('http://tianqi.2345.com/t/wea_history/js/%s0%s/56294_%s0%s.js' %(year,month,year,month))
            else:
                urls.append('http://tianqi.2345.com/t/wea_history/js/%s%s/56294_%s%s.js' %(year,month,year,month))
urls

In [ ]:

info = []
for url in urls:
    seconds = random.randint(3,6)
    response = requests.get(url, headers = headers).text
    ymd = re.findall("ymd:'(.*?)',",response)
    high = re.findall("bWendu:'(.*?)℃',",response)
    low = re.findall("yWendu:'(.*?)℃',",response)
    tianqi = re.findall("tianqi:'(.*?)',",response)
    fengxiang = re.findall("fengxiang:'(.*?)',",response)
    fengli = re.findall(",fengli:'(.*?)'",response)
    aqi = re.findall("aqi:'(.*?)',",response)
    aqiInfo = re.findall("aqiInfo:'(.*?)',",response)
    aqiLevel = re.findall(",aqiLevel:'(.*?)'",response)
    if len(aqi) == 0:
        aqi = None
        aqiInfo = None
        aqiLevel = None
        info.append(pd.DataFrame({'ymd':ymd,'high':high,'low':low,'tianqi':tianqi,'fengxiang':fengxiang,'fengli':fengli,'aqi':aqi,'aqiInfo':aqiInfo,'aqiLevel':aqiLevel}))
    else:
        info.append(pd.DataFrame({'ymd':ymd,'high':high,'low':low,'tianqi':tianqi,'fengxiang':fengxiang,'fengli':fengli,'aqi':aqi,'aqiInfo':aqiInfo,'aqiLevel':aqiLevel}))
    time.sleep(seconds)

# 生成数据表
weather = pd.concat(info)
# 数据导出
weather.to_csv('weather.csv',index = False)
print("all data ahve been saved!")

In [ ]:
# -*- coding:utf-8 -*-
import urllib
import re,codecs
import time,random
import requests
from lxml import html
from urllib import parse

In [ ]:
url="https://search.51job.com/list/090200,000000,0000,32,9,99,Java%25E5%25BC%2580%25E5%258F%2591,2,1.html"
headers={'user-agent':'*****'}

r= requests.get(url,headers=headers)
s=requests.session()
s.keep_alive = False
r.encoding = 'gbk'
# print(r.text)
reg = re.compile(r'"job_href":"(.*?)"',re.S)
ss=re.findall(reg, r.text)
ss = [item.replace("\\", "") for item in ss]
print(ss)
for link in ss:
    print(link)

现在我们处理一下之前所获得的招聘数据，想达到的目标如下：

1.去掉职位中的括号及集中内容

2.薪酬的格式不易统计，改成统一单位，并分割成最低薪酬与最高薪酬

3.将地区去掉区级单位，以便可视化统计

4.将学历要求中的招**人统一改为无学历要求

In [ ]:
import pandas as pd
import numpy as np
 
data=pd.read_excel('51job.xls',sheet_name='Sheet1',header=0)
df=pd.DataFrame(data)
df=df[True- df.公司.duplicated()]#去重
df=df[df.职位.str.contains(r'.*?数据.*?|.*?分析.*?')]#提取包含数据或者分析的岗位
df.to_excel('new51job.xls')

In [ ]:
# 1.去掉职位中的括号及集中内容
def get_position(position):
    #str1 = '数据分析师 (职位编号：6826227877739268360)'处理
    #去掉制表符
    temp1= re.sub('\t', '', position)
    #去掉括号中的内容，英文的括号要加反斜杠
    temp2 = re.sub('\(.*?\)','',temp1)
    #去掉括号中的内容，中文括号
    pos = re.sub('（.*?）', '', temp2)
    return pos

In [ ]:

#薪酬改成统一单位，并分割成最低薪酬与最高薪酬

#利用正则表达式提取月薪，把待遇规范成千/月的形式
def get_salary(salary):
    if '-'in salary: #针对1-2万/月或者10-20万/年的情况，包含-
        low_salary=re.findall(re.compile('(\d*\.?\d+)'),salary)[0]
        high_salary=re.findall(re.compile('(\d?\.?\d+)'),salary)[1]
        if u'万' in salary and u'年' in salary:#单位统一成千/月的形式
            low_salary = float(low_salary) / 12 * 10
            high_salary = float(high_salary) / 12 * 10
        elif u'万' in salary and u'月' in salary:
            low_salary = float(low_salary) * 10
            high_salary = float(high_salary) * 10
    else:#针对20万以上/年和100元/天这种情况，不包含-，取最低工资，没有最高工资
        high_salary=""
        if u'面议' in salary:
            low_salary = ""
        else:
            low_salary = re.findall(re.compile('(\d*\.?\d+)'), salary)[0]        
        if u'万' in salary and u'年' in salary:#单位统一成千/月的形式
            low_salary = float(low_salary) / 12 * 10
        elif u'万' in salary and u'月' in salary:
            low_salary = float(low_salary) * 10
        elif u'元'in salary and u'天'in salary:
            low_salary=float(low_salary)/1000*21#每月工作日21天
        
    return low_salary,high_salary

In [ ]:
#去掉地区的区级单位
def get_area(area):
    b = pd.Series(area)
    c = b.str.split('-')
    return c[0]

In [ ]:
#将学历要求中的招**人统一改为无学历要求
def get_edu(edu):
    #招**人替换成无学位要求
    edu = re.sub('招.*?人', '无学历要求',edu)
    return edu

In [ ]:
#coding:utf8
import xlrd
import codecs
import re
import pandas as pd

def load_from_xlsx(file):
    data = xlrd.open_workbook(file)
    table0 = data.sheet_by_name('51job')
    nrows = table0.nrows
    return table0, nrows
#利用正则表达式提取月薪，把待遇规范成千/月的形式
def get_salary(salary):
    if '-'in salary: #针对1-2万/月或者10-20万/年的情况，包含-
        low_salary=re.findall(re.compile('(\d*\.?\d+)'),salary)[0]
        high_salary=re.findall(re.compile('(\d?\.?\d+)'),salary)[1]
        if u'万' in salary and u'年' in salary:#单位统一成千/月的形式
            low_salary = float(low_salary) / 12 * 10
            high_salary = float(high_salary) / 12 * 10
        elif u'万' in salary and u'月' in salary:
            low_salary = float(low_salary) * 10
            high_salary = float(high_salary) * 10
    else:#针对20万以上/年和100元/天这种情况，不包含-，取最低工资，没有最高工资
        high_salary=""
        if u'面议' in salary:
            low_salary = ""
        else:
            low_salary = re.findall(re.compile('(\d*\.?\d+)'), salary)[0]        
        if u'万' in salary and u'年' in salary:#单位统一成千/月的形式
            low_salary = float(low_salary) / 12 * 10
        elif u'万' in salary and u'月' in salary:
            low_salary = float(low_salary) * 10
        elif u'元'in salary and u'天'in salary:
            low_salary=float(low_salary)/1000*21#每月工作日21天
        
    return low_salary,high_salary
 
def get_area(area):
    b = pd.Series(area)
    c = b.str.split('-')
    return c[0][0]
 
def get_position(position):
    #str1 = '数据分析师 (职位编号：6826227877739268360)'处理
    #去掉制表符
    temp1= re.sub('\t', '', position)
    #去掉括号中的内容，英文的括号要加反斜杠
    temp2 = re.sub('\(.*?\)','',temp1)
    #去掉括号中的内容，中文括号
    pos = re.sub('（.*?）', '', temp2)
    return pos
 
def get_edu(edu):
    #招**人替换成无学位要求
    edu = re.sub('招.*?人', '无学历要求',edu)
    return edu

def main():
    data = load_from_xlsx(r'51job.xls')
    table, nrows = data[0], data[1]
    print(f'一共有{nrows}行数据，开始清洗数据')
    for i in range(1,nrows):
        position = table.row_values(i)[1]
        company=table.row_values(i)[0]
        area = table.row_values(i)[3]#地区取到城市，把区域去掉
        salary=table.row_values(i)[2]
        companytype = table.row_values(i)[6]
        companyscale = table.row_values(i)[7]
        experience = table.row_values(i)[4]
        edu = table.row_values(i)[5]
        field = table.row_values(i)[8]
        if salary:#如果待遇这栏不为空，计算最低最高待遇
            getsalary=get_salary(salary)
            low_salary=getsalary[0]
            high_salary=getsalary[1]
        else:
            low_salary=high_salary=""
        area = get_area(area)
        position = get_position(position)
        edu = get_edu(edu)
#         print('正在写入第{}条'.format(i))
        output = '{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(position, company, area, low_salary, high_salary, companytype,edu,experience,companyscale, field)
        f=codecs.open('51job_clear.xls','a')  #unicode写入open方式打开的文件有问题，使用codecs，但是这里的xls本质是txt
        f.write(output) 
        f.close()
        #以下代码需要重新存储文件类型为xls后才能操作
    df = pd.read_excel('51job_clear.xls', engine='xlrd',header=None, names=['position', 'company', 'area', 'low_salary', 'high_salary', 'companytype', 'edu', 'experience', 'companyscale', 'field'])
    df.to_excel('51job_clear_name.xls',index=False)
if __name__=='__main__':
    main()
    print("数据清理完毕！")